<div style="background-color:grey; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

### Lab 3: Customized Memory Management Management

</div>

In [1]:
# Install the Google Generative AI library
#!pip install -q google-generativeai

import google.generativeai as genai
import os
import sys
import uuid
import json
import time
from IPython.display import display, Markdown
# Import the function from your helper file
from helper import get_gemini_api_key

def print_markdown(text):
    """Prints text as markdown in a notebook."""
    display(Markdown(text))

# --- Configuration ---
try:
    api_key = get_gemini_api_key()
    if not api_key:
        raise ValueError("API key is missing. Please ensure your helper.py file returns a valid key.")
    genai.configure(api_key=api_key)
    print("\nGemini API configured successfully!")
except Exception as e:
    print(f"An error occurred during configuration: {e}")

# Create the system prompt file for the task agent
with open("task_queue_system_prompt.txt", "w") as f:
    f.write("You are a task-management assistant. Your goal is to manage a list of tasks in your memory. When a user gives you tasks, use the `task_queue_push` tool for each one. When a user asks you to perform or complete tasks, use the `task_queue_pop` tool to work through them one by one. After managing tasks, provide a brief confirmation to the user. The user's name is Charles. You must always refer to him as Charles.")



Gemini API configured successfully!


/opt/anaconda3/envs/gl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Core Agent and Tool Logic
This section contains the reusable GeminiAgent class, the tool definitions and implementations, and the main run_agent_turn function that handles the interaction loop.

In [2]:
# --- Agent Class ---
class GeminiAgent:
    """A class to simulate an agent's state and memory."""
    def __init__(self, name, model_name, system_prompt="", memory_blocks=None, tools=None):
        self.id = f"agent-{uuid.uuid4()}"
        self.name = name
        self.memory_blocks = memory_blocks if memory_blocks else {}
        self.tools = tools if tools else []
        self.system_prompt = system_prompt
        self.model = genai.GenerativeModel(
            model_name=model_name,
            tools=self.tools
        )

    def get_formatted_memory(self):
        """Formats memory blocks into a string for the system prompt."""
        if not self.memory_blocks:
            return ""
        formatted_string = "--- CORE MEMORY ---\n"
        for label, value in self.memory_blocks.items():
            val_str = json.dumps(value) if isinstance(value, list) else str(value)
            formatted_string += f"<{label}>\n{val_str}\n</{label}>\n"
        return formatted_string.strip()

# --- Reusable Printing Function ---
def print_message(message_type, content):
    """Prints formatted messages based on their type."""
    if message_type == "reasoning":
        print(f"🧠 Reasoning: {content}")
    elif message_type == "assistant":
        print(f"🤖 Agent: {content}")
    elif message_type == "tool_call":
        tool_name = content.get("name", "N/A")
        arguments = content.get("arguments", {})
        print(f"🔧 Tool Call: {tool_name}\n{json.dumps(arguments, indent=2)}")
    elif message_type == "tool_return":
        print(f"🔧 Tool Return: {content}")
    elif message_type == "user":
        print(f"👤 User Message: {content}")
    else:
        print(content)
    print("-----------------------------------------------------")

# --- Tool Definitions (for the model) ---
def get_agent_id():
    """Query your agent ID field."""
    pass

def task_queue_push(task_description: str):
    """
    Push a task to a task queue stored in core memory.
    Args:
        task_description (str): A description of the next task you must accomplish.
    """
    pass

def task_queue_pop():
    """Get the next task from the task queue."""
    pass

# --- Tool Implementations (for Python) ---
def _get_agent_id_impl(agent: GeminiAgent):
    """Implementation for getting the agent ID."""
    return agent.id

def _task_queue_push_impl(agent: GeminiAgent, task_description: str):
    """Implementation for pushing a task."""
    if "tasks" not in agent.memory_blocks or not isinstance(agent.memory_blocks["tasks"], list):
        agent.memory_blocks["tasks"] = []
    tasks = agent.memory_blocks["tasks"]
    tasks.append(task_description)
    return f"Task '{task_description}' was added. Current tasks: {json.dumps(tasks)}"

def _task_queue_pop_impl(agent: GeminiAgent):
    """Implementation for popping a task."""
    if "tasks" not in agent.memory_blocks or not agent.memory_blocks["tasks"]:
        return "The task queue is empty."
    tasks = agent.memory_blocks["tasks"]
    popped_task = tasks.pop(0)
    return f"Completed task: '{popped_task}'. Remaining tasks: {json.dumps(tasks)}"

# --- Generic Agent Interaction Loop ---
def run_agent_turn(agent, tool_registry, user_message):
    """Handles a single turn of conversation with an agent, including tool calls."""
    print_message("user", user_message)
    full_prompt = f"{agent.system_prompt}\n\n{agent.get_formatted_memory()}\n\n**Task:**\n{user_message}"
    history = [{'role': 'user', 'parts': [{'text': full_prompt}]}]
    
    response = agent.model.generate_content(history, tools=agent.tools)
    time.sleep(1)
    message = response.candidates[0].content
    history.append(message)
    
    last_action_was_tool = False
    while any(part.function_call for part in message.parts):
        last_action_was_tool = True
        tool_response_parts = []
        for part in message.parts:
            if not part.function_call: continue
            fc = part.function_call
            tool_name = fc.name
            tool_args = dict(fc.args)
            print_message("reasoning", f"Model wants to call `{tool_name}`.")
            print_message("tool_call", {"name": tool_name, "arguments": tool_args})
            result = tool_registry[tool_name](**tool_args)
            print_message("tool_return", result)
            tool_response_parts.append({"function_response": {"name": tool_name, "response": {"content": result}}})
        
        history.append({"role": "tool", "parts": tool_response_parts})
        response = agent.model.generate_content(history, tools=agent.tools)
        time.sleep(1)
        message = response.candidates[0].content
        history.append(message)

    if last_action_was_tool:
        print_message("reasoning", "All tools executed. Generating final summary.")
        history.append({'role': 'user', 'parts': [{'text': "All tasks are complete. Please provide a final, summary response, including any creative content you were asked to generate."}]})
        final_response = agent.model.generate_content(history)
        final_text = final_response.candidates[0].content.parts[0].text
    else:
        final_text = message.parts[0].text if message.parts else "Task complete."

    print_message("assistant", final_text)


### 3. Section 1: Memory Blocks
This block creates the first agent and demonstrates how to initialize and view its memory_blocks.

In [3]:
# --- 1. Create an agent with initial memory ---
agent1 = GeminiAgent(
    name="agent1",
    model_name="gemini-1.5-flash",
    memory_blocks={
        "human": "The human's name is Bob the Builder.",
        "persona": "My name is Sam, the all-knowing sentient AI."
    }
)

# --- 2. Accessing memory blocks ---
print("All memory blocks:")
print(json.dumps(agent1.memory_blocks, indent=2))

print("\nFormatted memory prompt:")
print(agent1.get_formatted_memory())


All memory blocks:
{
  "human": "The human's name is Bob the Builder.",
  "persona": "My name is Sam, the all-knowing sentient AI."
}

Formatted memory prompt:
--- CORE MEMORY ---
<human>
The human's name is Bob the Builder.
</human>
<persona>
My name is Sam, the all-knowing sentient AI.
</persona>


### 4. Section 2: Accessing Agent State with Tools
Here, we create a second agent and give it the get_agent_id tool to allow it to access its own state.

In [4]:
# --- 1. Create an agent with the get_id tool ---
agent2 = GeminiAgent(
    name="agent2",
    model_name="gemini-1.5-flash",
    tools=[get_agent_id]
)

# --- 2. Setup the tool registry ---
tool_registry_2 = {
    "get_agent_id": lambda: _get_agent_id_impl(agent2)
}

# --- 3. Run the agent ---
run_agent_turn(
    agent=agent2,
    tool_registry=tool_registry_2,
    user_message="What is your agent id?"
)


👤 User Message: What is your agent id?
-----------------------------------------------------
🧠 Reasoning: Model wants to call `get_agent_id`.
-----------------------------------------------------
🔧 Tool Call: get_agent_id
{}
-----------------------------------------------------
🔧 Tool Return: agent-c5534ed9-1ca1-45fe-8d66-9ab8ece98bab
-----------------------------------------------------


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 11
}
]

### 5. Section 3: Custom Task Queue Memory
This final section creates the more advanced task_agent that uses tools to modify its own memory, creating a simple task queue.

In [9]:
# --- 1. Create the Task Agent ---
task_agent_system_prompt = open("task_queue_system_prompt.txt", "r").read()

task_agent = GeminiAgent(
    name="task_agent",
    model_name="gemini-1.5-flash",
    system_prompt=task_agent_system_prompt,
    memory_blocks={
        "tasks": []
    },
    tools=[task_queue_push, task_queue_pop]
)

# --- 2. Setup Tool Registry ---
task_tool_registry = {
    "task_queue_push": lambda task_description: _task_queue_push_impl(task_agent, task_description),
    "task_queue_pop": lambda: _task_queue_pop_impl(task_agent)
}

# --- 3. Use the Task Agent ---
print("Initial task list:", json.dumps(task_agent.memory_blocks["tasks"]))

# Add two tasks
run_agent_turn(
    agent=task_agent,
    tool_registry=task_tool_registry,
    user_message="Add 'start calling me Charles' and 'tell me a haiku about my name' as two separate tasks."
)

print("Task list after adding:", json.dumps(task_agent.memory_blocks["tasks"], indent=2))

# Complete the tasks
run_agent_turn(
    agent=task_agent,
    tool_registry=task_tool_registry,
    user_message="Okay, please complete your tasks now."
)

print("Final task list:", json.dumps(task_agent.memory_blocks["tasks"]))


Initial task list: []
👤 User Message: Add 'start calling me Charles' and 'tell me a haiku about my name' as two separate tasks.
-----------------------------------------------------
🧠 Reasoning: Model wants to call `task_queue_push`.
-----------------------------------------------------
🔧 Tool Call: task_queue_push
{
  "task_description": "start calling me Charles"
}
-----------------------------------------------------
🔧 Tool Return: Task 'start calling me Charles' was added. Current tasks: ["start calling me Charles"]
-----------------------------------------------------
🧠 Reasoning: Model wants to call `task_queue_push`.
-----------------------------------------------------
🔧 Tool Call: task_queue_push
{
  "task_description": "tell me a haiku about my name"
}
-----------------------------------------------------
🔧 Tool Return: Task 'tell me a haiku about my name' was added. Current tasks: ["start calling me Charles", "tell me a haiku about my name"]
---------------------------------